In [1]:
!pip install -q torchviz

In [2]:
import torch
import matplotlib.pyplot as plt

# 1. Automatic Differentiation with torch.autograd

Before proceeding autograd, will understand the basic terms:

- Forward Propagation:
    Computes the model's output by passing the input data through the network layers. It is often called Forward pass.

- Backward Propagation:
    Calculates the gradients of the loss with respect to the model's parameters using the chain rule, enabling parameter updates to minimize the loss.



### 1.1. torch.autograd

[Autograd docs](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#a-gentle-introduction-to-torch-autograd)

- We create two tensors x and y with requires_grad=True, indicating that we want to compute gradients for these tensors.

- We perform simple operations on x and y to obtain z.

- Computing Gradients: We call z.backward() to compute the gradients of z with respect to x and y. The gradients are stored in the grad attribute of each tensor.

In the following example:

- The operation is z=x⋅y+y2

- The partial derivative of z with respect to x is ∂z∂x=y

- The partial derivative of z with respect to y is ∂z∂y=x+2y

Given x=2.0 and y=3.0:

- The gradient of z w.r.t. x is 3.0

- The gradient of z w.r.t. y is 2.0+2⋅3.0=8.0

Tensors that require gradients will have their operations tracked by PyTorch's autograd engine, enabling the computation of gradients during backpropagation.

![Computation Graph Through Autograd](media/Autograd-Computation-Graph-2-2.png)

The automatic differentiation provided by torch.autograd simplifies this process by handling the complex chain rule calculations needed for backpropagation through the entire network.

For ∂z/∂x:  
```math
\dfrac{∂z}{∂x} = \dfrac{∂z}{∂p} \dfrac{∂p}{∂x} + \dfrac{∂z}{∂q} \dfrac{∂q}{∂x} = 1⋅y+1⋅0 = y
```

For ∂z/∂y:
```math
\dfrac{∂z}{∂y} = \dfrac{∂z}{∂p} \dfrac{∂p}{∂y} + \dfrac{∂z}{∂q} \dfrac{∂q}{∂y} = 1⋅x+1⋅2y = x+2y
```

These equations correspond to the chain rule calculations happening behind the scenes, demonstrating how PyTorch's autograd system computes gradients through the computational graph.

In [3]:
# Create tensors with requires_grad=True
x = torch.tensor([2.0, 5.0], requires_grad=True)
y = torch.tensor([3.0, 7.0], requires_grad=True)

# Perform some operations
z = x * y + y**2

z.retain_grad() #By default intermediate layer weight updation is not shown.

# Compute the gradients
z_sum = z.sum().backward()


print(f"Gradient of x: {x.grad}")
print(f"Gradient of y: {y.grad}")
print(f"Gradient of z: {z.grad}")
print(f"Result of the operation: z = {z.detach()}")

Gradient of x: tensor([3., 7.])
Gradient of y: tensor([ 8., 19.])
Gradient of z: tensor([1., 1.])
Result of the operation: z = tensor([15., 84.])


### 1.2. Gradient Computation Graph

A computation graph is a visual representation of the sequence of operations performed on tensors in a neural network, showing how each operation contributes to the final result. It is crucial for understanding and debugging the flow of data and gradients in deep learning models.

torchviz is a tool used to visualize the computation graph of any PyTorch model.

![Operations Graph](media/Autograd-Operators-Graph-1-1.png)

In [4]:
from torchviz import make_dot

# Visualize the computation graph
dot = make_dot(z, params={"x": x, "y": y, "z" : z})
dot.render("grad_computation_graph", format="png")

'grad_computation_graph.png'

### 1.3. Detaching Tensors from Computation Graph

The detach() method is used to create a new tensor that shares storage with the original tensor but without tracking operations. When you call detach(), it returns a new tensor that does not require gradients. This is useful when you want to perform operations on a tensor without affecting the computation graph.


In [5]:
# Let's detach z from the computation graph
print("Before detaching z from computation: ", z.requires_grad)
z_det = z.detach()
print("After detaching z from computation: ", z_det.requires_grad)

Before detaching z from computation:  True
After detaching z from computation:  False


### 1.4. Can Backpropagation be performed when requires_grad=False?

Now the same tensors x and y are created with requires_grad=False.

When attempting to compute the gradients using z.backward(), a RuntimeError is raised because the tensors do not require gradients, and thus do not have a grad_fn.

In this case, since requires_grad=False was used, the computation graph is essentially empty, as no gradients will be tracked.

In [6]:
x = torch.tensor(2.0, requires_grad=False)
y = torch.tensor(3.0, requires_grad=False)


# Perform simple operations
z = x * y + y**2


# Compute the gradients
z.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

# 2. Backpropagation in Neural Networks

![Gradient Descent](media/gradient-descent-2d-diagram.png)

We have understood the mathematical intuition behind and how torch.autograd takes care of automatic differentiation with an example.

Then,

- The loss is calculate between prediction and target using loss(predcition,target)
- Then backpropagation is performed using loss.backward()
- We update the new weights using optimizer.step()

![keras-linear-regression-weight-update-block-diagram](media/keras-linear-regression-weight-update-block-diagram.png)

The Weight Update Formula is:

```math
w_{new} = w - \eta \nabla L(w)
```

The gradient of L is a vector of partial derivatives:

```math
\nabla L = \left(\dfrac{∂L}{∂w_1}, \dfrac{∂L}{∂w_2}, ... , \dfrac{∂L}{∂w_n}\right)
```

- Current Weights (w): These are the weights of the model before the update.
- Learning Rate (η): A hyperparameter that controls the step size of the weight update. A smaller learning rate makes the training process slower but more precise, while a larger learning rate makes the training process faster but it may sometimes overshoot the optimal solution.
- Gradient of the Loss (∇L(w)): The partial derivatives of the loss function with respect to each weight. This indicates the direction and magnitude of the steepest ascent in the loss function.
- Updated Weights (wnew): The new weights after applying the gradient descent step.

After applying the updates, it's crucial to zero out the gradients. This is typically done using: optimizer.zer_grad()

Interesting, right? This is how the model learns. During training, the model iteratively adjusts its weights and biases to minimize the loss function, which measures the difference between the predicted outputs and the actual targets.

This process of forward pass, gradient computation, backward pass, and parameter update is repeated for many iterations (epochs) over the entire dataset. With each iteration, the model's parameters are refined, gradually reducing the loss and improving the model's performance.

In our upcoming notebooks we will understand all these concepts with hands on training.



# 3. Conclusion

We learnt how the gradients are updated mathematically with a node graph diagrams. We also visualized the computation graph using torchviz, providing a clear understanding of how operations are structured in PyTorch. Finally, we understood how weight updates happen during training with code, illustrating the core mechanism of model learning.

While this notebook provides a great start, our full course offers detailed explanations, hands-on projects, and expert guidance to help you master PyTorch and deep learning. Join here: [Deep Learning with PyTorch Course](https://opencv.org/university/deep-learning-with-pytorch/)
